## Problem 5

In [1]:
 # Import packages
import numpy as np
import pandas as pd
import scipy.stats as sts
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')
from scipy import special
from scipy.integrate import quad

In [2]:
# Load data
data = np.loadtxt('data/MacroSeries.txt', delimiter=",")
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]

In [3]:
def data_moments(c, k):
    mean_c = c.mean()
    mean_k = k.mean()
    var_c = c.var()
    var_k = k.var()
    corr_ck = np.corrcoef(c, k)[0, 1]
    corr_k = np.corrcoef(k[:-1], k[1:])[0, 1]
    
    return mean_c, mean_k, var_c, var_k, corr_ck, corr_k

meank = k.mean()
def simulate_moments(alpha, beta, rho, mu, sigma, T=100, S=1000):
    z1 = mu
    k1 = meank
    epsilon = np.random.normal(0, scale=sigma, size=(S, T)) # Simulate shocks
    zmat = np.zeros((S, T))
    kmat = np.zeros((S, T))
    cmat = np.zeros((S, T))
    zmat[:, 0] = z1 * np.ones(S)
    kmat[:, 0] = k1 * np.ones(S)
    
    for i in range(1, T):
        zmat[:, i] = rho * zmat[:, i-1] + (1 - rho) * mu + epsilon[:, i]
        kmat[:, i] = alpha * beta * np.exp(zmat[:, i]) + kmat[:, i-1] ** alpha
    wmat = (1 - alpha) * np.exp(zmat) * kmat ** alpha
    rmat = alpha * np.exp(zmat) * kmat ** (alpha - 1)
    
    cmat[:, 0] = wmat[:, 0] + rmat[:, 0] * kmat[:, 0]
    for i in range(1, T):
        cmat[:, i] = wmat[:, i] + rmat[:, i] * kmat[:, i] - kmat[:, i-1]
    
    smean_c = cmat.mean(axis=1).mean()
    smean_k = kmat.mean(axis=1).mean()
    svar_c = cmat.var(axis=1).mean()
    svar_k = kmat.var(axis=1).mean()
    corr_ck = np.zeros(T)
    corr_k = np.zeros(T - 1)
    
    for i in range(T):
        corr_ck[i] = np.corrcoef(cmat[:, i], kmat[:, i])[0, 1]
    for i in range(T-1):
        corr_k[i] = np.corrcoef(kmat[:-1, i], kmat[1:, i])[0, 1] 
    scorr_ck = corr_ck[2:].mean()
    scorr_k = corr_k.mean()
    
    return smean_c, smean_k, svar_c, svar_k, scorr_ck, scorr_k

def err_vec(c, k, w, r, alpha, beta, rho, mu, sigma, simple = False):
    mo_moments = np.array(simulate_moments(alpha, beta, rho, mu, sigma))
    da_moments = np.array(data_moments(c, k))
    if simple:
        err_vec = (mo_moments - da_moments)
    else:
        err_vec = ((mo_moments - da_moments) / da_moments)
    return err_vec

def crit(params, *args):
    alpha, beta, rho, mu, sigma = params
    c, k, w, r, weight  = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu, sigma)
    crit_val = np.dot(np.dot(err.T, weight), err) 
    
    return crit_val

In [10]:
weight = np.identity(6)
alpha_init = 0.8
beta_init = 0.9
rho_init = 0.7
mu_init = 0.2
sigma_init = 0.2

params_init = (alpha_init, beta_init, rho_init, mu_init, sigma_init)
bnds = ((0.01, 0.99), (0.01, 0.99), (-0.99, 0.99), (-0.5, 1), (0.001, 1))
args = (c, k, w, r, weight)
results = opt.minimize(crit, params_init, args=(args), bounds = bnds, method = 'L-BFGS-B')
print(results)

      fun: 4.4442414009306894
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1167253.15913069,  -499840.20774616,   493214.2289519 ,
       -1595457.87893709,   978780.18585389])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 306
      nit: 5
   status: 0
  success: True
        x: array([ 0.80000024,  0.90000011,  0.69999787,  0.19999912,  0.20000101])


It seems like the function converges most of the time, to the initial guesses...